**Plants Dataset Training**

In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import pandas as pd

# Assuming our CSV file is named 'PlantCLEF2022 train metadata.csv'
csv_file_path = '/content/drive/MyDrive/CLEF2022/filtered_data.csv'

# Read the CSV file
csv_data = pd.read_csv(csv_file_path)

print(csv_data)

       classid                                    image_name  \
0      9848396  0966faf193bf90fb66cbd220c5738eac5b4e5961.jpg   
1      9717715  078ccd025ca2c5bb5bcdf0320f3f8a19235a3272.jpg   
2      9717715  a0371fdc1c5b116e6e58252893abdf51aa1ad61a.jpg   
3      9717715  5d1261c34a6ce7d2ab6873057c36bc4ac389329f.jpg   
4      9717715  91501758f741a2d7e4439946044fb0ad80639f23.jpg   
...        ...                                           ...   
44226  9765658  3e17daea73ff1c7ef2c5989f13a70c1dcd977e58.jpg   
44227  9765658  d964ab3dbdcbb6d228c03fa8af25b30f19ff572a.jpg   
44228  9765658  fb6ebbc712d6ed8ab940d1a7bf0d31f450d2313f.jpg   
44229  9765658  e23f4f4ca37b4d8ea9da09db923815822ccbb47f.jpg   
44230  9765658  0f34ed9d4707e7b8cd6997cb2993afe2536d663e.jpg   

                                              image_path  \
0      9848396/0966faf193bf90fb66cbd220c5738eac5b4e59...   
1      9717715/078ccd025ca2c5bb5bcdf0320f3f8a19235a32...   
2      9717715/a0371fdc1c5b116e6e58252893abdf51aa1a

In [ ]:
# Path to the main image folder
main_folder_path = "/content/drive/MyDrive/CLEF2022/PlantCLEF2022_trusted_training_images_9/images"

In [ ]:
#!ls "/content/drive/MyDrive/CLEF2022/PlantCLEF2022_trusted_training_images_9/images"

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
# Load filtered data
csv_data = pd.read_csv('/content/drive/MyDrive/CLEF2022/filtered_data.csv')
# Assuming 'class' column contains labels
#labels = csv_data['classid']
#labels = csv_data ['image_path']
#labels = csv_data ['image_name']
#labels = csv_data ['family']
#labels = csv_data ['species']
#print(labels)
labels = csv_data[['classid', 'image_path', 'image_name', 'family', 'species']]

labels.info()

 #Extract the 'species' column for label encoding
labels = csv_data['species']

# Create a LabelEncoder object
encoder = LabelEncoder()

# Apply label encoding to the 'species' column
encoded_labels = encoder.fit_transform(labels)
# Create a new DataFrame with encoded labels
encoded_df = pd.DataFrame({'species': encoded_labels})

# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    csv_data['image_path'], labels, test_size=0.2, random_state=42
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44231 entries, 0 to 44230
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   classid     44231 non-null  int64 
 1   image_path  44231 non-null  object
 2   image_name  44231 non-null  object
 3   family      44231 non-null  object
 4   species     44231 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [ ]:
# Create image generators for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to the range 0-1
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True  # Randomly flip images horizontally
)

val_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to the range 0-1
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True  # Randomly flip images horizontally
)


In [ ]:
# Iterate over subfolders and create generators for each class
for subfolder in os.listdir(main_folder_path):
  subfolder_path = os.path.join(main_folder_path, subfolder)
  #print(subfolder_path)

  # Only create generators for folders containing image files
  if os.path.isdir(subfolder_path) and any(file.endswith(('.jpg', '.jpeg', '.png')) for file in os.listdir(subfolder_path)):
    class_name = subfolder  # Assuming subfolder names represent class labels
    #print(class_name)
    # Filter CSV data for the current class
    #csv_data['classid'] = csv_data['classid'].astype(str)
    filter_data = csv_data[csv_data['image_path'].str.contains(subfolder)]
    #print(filter_data)

     #[csv_data['classid'] == class_name]
     #subfolder names(classes)
   # print(f"Class Name: {class_name}")
    # Set display options to show all columns and the desired number of rows
    #pd.set_option('display.max_columns', None)

    #print(filter_data.head(5).to_string())
   # print(f"Number of Rows: {filter_data.shape}")

    #print(filter_data.info())
    # for column names
    #print(filter_data.keys())
    #for show rows
    #print(filter_data.shape[0])




In [ ]:
"classid" in csv_data.columns

True

In [ ]:
csv_data["classid"].isnull().sum()

0

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
  dataframe=csv_data,
  directory=main_folder_path,
  x_col="image_path",
  y_col="species",
  target_size=(256, 256),
  batch_size=64,
  class_mode="categorical",
  subset="training"
)

val_generator = val_datagen.flow_from_dataframe(
  dataframe=filter_data,
  directory=main_folder_path,
  x_col="image_path",
  y_col="species",
  target_size=(256, 256),
  batch_size=64,
  class_mode = "categorical",
  subset="validation",
  validation_data=encoded_df
)
# Define the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # replace num_classes with actual number
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(train_generator, epochs=10)

# Evaluate the model
#model.evaluate(val_generator)







Found 44231 validated image filenames belonging to 1397 classes.
Found 0 validated image filenames belonging to 1 classes.


In [ ]:
class_data.columns

Index(['classid', 'image_name', 'image_path', 'species', 'genus', 'family',
       'order', 'class', 'source', 'manual_tag', 'predicted_tag',
       'predicted_tag_probability', 'original_url', 'license', 'publisher',
       'gbif_occurrence_id', 'aggregator', 'dataset_key', 'image_backup_url',
       'absolute_path'],
      dtype='object')